In [2]:
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
import scipy.stats as stats
from tqdm import tqdm

In [3]:
def generate_uniform(N):
    return np.random.uniform(-1,1,N)

def generate_3uniform(N):
    return np.random.uniform(-1,1,N) + np.random.uniform(-1,1,N) + np.random.uniform(-1,1,N)

def generate_cauchy(N):
    return np.random.standard_cauchy(N)

def generate_normal(N):
    return np.random.normal(0, 1, int(N))

# 1

In [27]:
qs = [0.9, 0.95, 0.99]
ps = [0.01, 0.95]
for p in ps:
    for q in qs:
        z = stats.norm.ppf((1+q)/2)
        if p < 0.5:
            n = ((np.sqrt(1-p) * z + np.sqrt((1-p) * z * z + 4)) / (2 * np.sqrt(p)))**2
        else:
            n = ((np.sqrt(p) * z + np.sqrt((p) * z * z + 4)) / (2 * np.sqrt(1-p)))**2
        n = math.ceil(n)
        print(f"p={p}, Q={q} : n={n}")
        l, r = (n*p - np.sqrt(n*p*(1-p)) * z, n*p + np.sqrt(n*p*(1-p)) * z)
        print(f"        Границы доверительного интервала: {math.floor(l), math.ceil(r)}")

p=0.01, Q=0.9 : n=446
        Границы доверительного интервала: (1, 8)
p=0.01, Q=0.95 : n=563
        Границы доверительного интервала: (1, 11)
p=0.01, Q=0.99 : n=846
        Границы доверительного интервала: (1, 16)
p=0.95, Q=0.9 : n=87
        Границы доверительного интервала: (79, 86)
p=0.95, Q=0.95 : n=110
        Границы доверительного интервала: (100, 109)
p=0.95, Q=0.99 : n=164
        Границы доверительного интервала: (148, 163)


# 2

In [8]:
def confidence_interval(p=None, q=0.95, data=None):
    """
    Рассчитывает доверительный интервал для математического ожидания или дисперсии в зависимости от переданных параметров.
    
    :param p: Уровень значимости для математического ожидания
    :param q: Уровень доверия
    :param data: Массив данных, если нужен интервал для дисперсии
    :return: Границы интервала (левая и правая)
    """
    if data is not None:
        # Доверительный интервал для дисперсии
        N = len(data)
        sample_var = np.var(data, ddof=1)  # Оценка дисперсии с исправлением на степень свободы
        alpha = 1 - q

        # Квантили хи-квадрат распределения
        chi2_left = stats.chi2.ppf(1 - alpha / 2, N - 1)
        chi2_right = stats.chi2.ppf(alpha / 2, N - 1)

        # Границы доверительного интервала для дисперсии
        l = (N - 1) * sample_var / chi2_left
        r = (N - 1) * sample_var / chi2_right
        return l, r
    else:
        # Доверительный интервал для математического ожидания
        z = stats.norm.ppf((1 + q) / 2)
        
        if p is None:
            raise ValueError("Parameter 'p' is required for mean confidence interval.")
        
        if p < 0.5:
            n = ((np.sqrt(1 - p) * z + np.sqrt((1 - p) * z**2 + 4)) / (2 * np.sqrt(p)))**2
        else:
            n = ((np.sqrt(p) * z + np.sqrt((p) * z**2 + 4)) / (2 * np.sqrt(1 - p)))**2
            
        n = math.ceil(n)
        l = n * p - np.sqrt(n * p * (1 - p)) * z
        r = n * p + np.sqrt(n * p * (1 - p)) * z
        return math.floor(l), math.ceil(r)

In [9]:
def linearization_interval(confidence_interval_func, error=0.01, **kwargs):
    """
    Применяет метод линеаризации для доверительного интервала.

    :param confidence_interval_func: Функция для расчета доверительного интервала
    :param error: Относительная погрешность
    :param kwargs: Дополнительные аргументы, необходимые для confidence_interval_func
    :return: (начальные границы, границы с погрешностью, погрешности по каждой границе)
    """
    l, r = confidence_interval_func(**kwargs)

    if 'data' in kwargs:
        # Линеаризация для дисперсии
        N = len(kwargs['data'])
        sample_var = np.var(kwargs['data'], ddof=1)
        delta_var = error * sample_var

        # Границы с учетом линеаризации
        dl = -(N - 1) / stats.chi2.ppf(1 - (1 - kwargs['q']) / 2, N - 1) * delta_var
        dr = (N - 1) / stats.chi2.ppf((1 - kwargs['q']) / 2, N - 1) * delta_var
    else:
        # Линеаризация для математического ожидания
        p = kwargs['p']
        q = kwargs['q']
        z = stats.norm.ppf((1 + q) / 2)
        n = math.ceil(((np.sqrt(1 - p) * z + np.sqrt((1 - p) * z**2 + 4)) / (2 * np.sqrt(p)))**2) if p < 0.5 else \
            math.ceil(((np.sqrt(p) * z + np.sqrt((p) * z**2 + 4)) / (2 * np.sqrt(1 - p)))**2)

        delta_n = error * n
        dl = -z * np.sqrt((1 - p) / n) * delta_n
        dr = z * np.sqrt((1 - p) / n) * delta_n

    return (l, r), (l + dl, r + dr), (dl, dr)

In [10]:
def kreinovich_interval(confidence_interval_func, uncertainty=0.01, **kwargs):
    """
    Применяет метод Крейновича для доверительного интервала.

    :param confidence_interval_func: Функция для расчета доверительного интервала
    :param uncertainty: Относительная неопределенность
    :param kwargs: Дополнительные аргументы, необходимые для confidence_interval_func
    :return: (начальные границы, границы с учетом неопределенности, погрешности по каждой границе)
    """
    l, r = confidence_interval_func(**kwargs)

    if 'data' in kwargs:
        # Метод Крейновича для дисперсии
        N = len(kwargs['data'])
        sample_var = np.var(kwargs['data'], ddof=1)

        # Максимальные и минимальные значения дисперсии с учетом неопределенности
        sample_var_min = sample_var * (1 - uncertainty)
        sample_var_max = sample_var * (1 + uncertainty)

        # Границы с учетом максимальной и минимальной дисперсий
        l_min = (N - 1) * sample_var_min / stats.chi2.ppf(1 - (1 - kwargs['q']) / 2, N - 1)
        r_max = (N - 1) * sample_var_max / stats.chi2.ppf((1 - kwargs['q']) / 2, N - 1)
    else:
        # Метод Крейновича для математического ожидания
        p = kwargs['p']
        q = kwargs['q']
        z = stats.norm.ppf((1 + q) / 2)
        n = math.ceil(((np.sqrt(1 - p) * z + np.sqrt((1 - p) * z**2 + 4)) / (2 * np.sqrt(p)))**2) if p < 0.5 else \
            math.ceil(((np.sqrt(p) * z + np.sqrt((p) * z**2 + 4)) / (2 * np.sqrt(1 - p)))**2)

        n_min = n * (1 - uncertainty)
        n_max = n * (1 + uncertainty)

        l_min = n_min * p - np.sqrt(n_min * p * (1 - p)) * z
        r_max = n_max * p + np.sqrt(n_max * p * (1 - p)) * z

    return (l, r), (math.floor(l_min), math.ceil(r_max)), (l - l_min, r_max - r)

In [11]:
qs = [0.9, 0.95, 0.99]
ps = [0.01, 0.95]

for p in ps:
    for q in qs:
        print(f"=== p={p}, q={q} ===")
        
        # Начальный доверительный интервал
        n, l, r = calculate_confidence_interval_mean(p, q)
        print(f"Стандартный интервал: n={n}, границы ({l}, {r})")
        
        # Метод линеаризации
        lin_result = linearization_mean_interval(p, q)
        print(f"Линеаризация: Интервал с учетом погрешности ({lin_result[1][0]}, {lin_result[1][1]})")
        print(f"Погрешность левой и правой границ: {lin_result[2]}")
        
        # Метод Крейновича
        kreinovich_result = kreinovich_mean_interval(p, q)
        print(f"Крейнович: Интервал с учетом неопределенности ({kreinovich_result[1][0]}, {kreinovich_result[1][1]})")
        print(f"Погрешность левой и правой границ: {kreinovich_result[2]}\n")

=== p=0.01, q=0.9 ===
Стандартный интервал: n=446, границы (1, 8)
Линеаризация: Интервал с учетом погрешности (0.6543693218591138, 8.345630678140886)
Погрешность левой и правой границ: (-0.3456306781408862, 0.3456306781408862)
Крейнович: Интервал с учетом неопределенности (0, 8)
Погрешность левой и правой границ: (0.02358182628823391, -0.021854673840427274)

=== p=0.01, q=0.95 ===
Стандартный интервал: n=563, границы (1, 11)
Линеаризация: Интервал с учетом погрешности (0.5372782798603156, 11.462721720139685)
Погрешность левой и правой границ: (-0.4627217201396844, 0.4627217201396844)
Крейнович: Интервал с учетом неопределенности (0, 11)
Погрешность левой и правой границ: (0.030322984153512422, -0.6634042654050631)

=== p=0.01, q=0.99 ===
Стандартный интервал: n=846, границы (1, 16)
Линеаризация: Интервал с учетом погрешности (0.25454770110896374, 16.745452298891035)
Погрешность левой и правой границ: (-0.7454522988910363, 0.7454522988910363)
Крейнович: Интервал с учетом неопределенност

In [13]:
data = np.random.normal(0, 1, 100)

qs = [0.9, 0.95, 0.99]
ps = [0.01, 0.95]

for p in ps:
    for q in qs:
        print(f"=== p={p}, q={q} ===")
        
        # Интервал для математического ожидания
        standard_interval = confidence_interval(p=p, q=q)
        lin_result = linearization_interval(confidence_interval, p=p, q=q)
        kreinovich_result = kreinovich_interval(confidence_interval, p=p, q=q)
        print(f"Матем. ожидание — Стандартный интервал: {standard_interval}")
        print(f"Линеаризация: {lin_result[1]}, погрешности: {lin_result[2]}")
        print(f"Крейнович: {kreinovich_result[1]}, погрешности: {kreinovich_result[2]}\n")
        
        # Интервал для дисперсии
        standard_interval_var = confidence_interval(q=q, data=data)
        lin_result_var = linearization_interval(confidence_interval, q=q, data=data)
        kreinovich_result_var = kreinovich_interval(confidence_interval, q=q, data=data)
        print(f"Дисперсия — Стандартный интервал: {standard_interval_var}")
        print(f"Линеаризация: {lin_result_var[1]}, погрешности: {lin_result_var[2]}")
        print(f"Крейнович: {kreinovich_result_var[1]}, погрешности: {kreinovich_result_var[2]}\n")

=== p=0.01, q=0.9 ===
Матем. ожидание — Стандартный интервал: (1, 8)
Линеаризация: (0.6543693218591138, 8.345630678140886), погрешности: (-0.3456306781408862, 0.3456306781408862)
Крейнович: (0, 8), погрешности: (0.02358182628823391, -0.021854673840427274)

Дисперсия — Стандартный интервал: (0.8347273576581457, 1.3350338817231322)
Линеаризация: (0.8263800840815642, 1.3483842205403636), погрешности: (-0.008347273576581457, 0.013350338817231319)
Крейнович: (0, 2), погрешности: (0.008347273576581493, 0.013350338817231133)

=== p=0.01, q=0.95 ===
Матем. ожидание — Стандартный интервал: (1, 11)
Линеаризация: (0.5372782798603156, 11.462721720139685), погрешности: (-0.4627217201396844, 0.4627217201396844)
Крейнович: (0, 11), погрешности: (0.030322984153512422, -0.6634042654050631)

Дисперсия — Стандартный интервал: (0.8009490009212541, 1.4020985409757596)
Линеаризация: (0.7929395109120415, 1.416119526385517), погрешности: (-0.008009490009212542, 0.014020985409757594)
Крейнович: (0, 2), погрешн